# Seleccionar idioma

In [1]:
idioma = 'english'

# Importar Librerias

In [2]:
from datetime import date

today = date.today()
print("Today's date:", today)

Today's date: 2021-04-03


In [3]:
import nltk; 
#nltk.download('stopwords')
import time


In [4]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
from string import digits
from string import punctuation
import unidecode


In [5]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [6]:
import spacy


In [7]:
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

c:\users\gonza\tesisenv\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

c:\users\gonza\tesisenv\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words(idioma)
stop_words.extend(['linkremoved','amp', 'usernameremoved','link','removed', '<usernameremoved>','<linkremoved>','usernameremoved_usernameremoved','linkremoved_linkremoved'])
##stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [11]:
#stopwords

# Importar datos

In [12]:
df = pd.read_csv('../Airline tweets/airlines_tweets_with_sentiment_data.csv',encoding='utf-8')

In [13]:
len(df)

10560

In [14]:
df.head()

,Unnamed: 0,created_at,tweet_id,id_str,text_x,truncated,entities,source,in_reply_to_status_id,in_reply_to_status_id_str,...,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count_y,text_y,tweet_coord,tweet_created,tweet_location,user_timezone
0,0,Tue Feb 24 17:04:10 +0000 2015,570267956648792064,570267956648792064,@VirginAmerica you know what would be amazingl...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",NaN,NaN,...,Virgin America,NaN,JNLpierce,NaN,0,@VirginAmerica you know what would be amazingl...,NaN,2015-02-24 09:04:10 -0800,Boston | Waltham,Quito
1,1,Tue Feb 24 16:49:01 +0000 2015,570264145116819457,570264145116819457,@VirginAmerica I love this graphic. http://t.c...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,...,Virgin America,NaN,DT_Les,NaN,0,@VirginAmerica I love this graphic. http://t.c...,"[40.74804263, -73.99295302]",2015-02-24 08:49:01 -0800,NaN,NaN
2,2,Tue Feb 24 00:08:07 +0000 2015,570012257549070337,570012257549070337,@VirginAmerica I'm #elevategold for a good rea...,False,"{'hashtags': [{'text': 'elevategold', 'indices...","<a href=""http://twitter.com/download/iphone"" r...",5.699362e+17,5.699362e+17,...,Virgin America,NaN,arieldaie,NaN,0,@VirginAmerica I'm #elevategold for a good rea...,NaN,2015-02-23 16:08:07 -0800,Los Angeles,NaN
3,3,Tue Feb 24 19:13:57 +0000 2015,570300616901320704,570300616901320704,"@VirginAmerica yes, nearly every time I fly VX...",False,"{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""https://about.twitter.com/products/tw...",5.702999e+17,5.702999e+17,...,Virgin America,NaN,cjmcginnis,NaN,0,"@VirginAmerica yes, nearly every time I fly VX...",NaN,2015-02-24 11:13:57 -0800,San Francisco CA,Pacific Time (US & Canada)
4,4,Mon Feb 23 21:30:10 +0000 2015,569972508499283968,569972508499283968,@VirginAmerica Congrats on winning the @Travel...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://www.hootsuite.com"" rel=""nofoll...",NaN,NaN,...,Virgin America,NaN,Travelzoo,NaN,0,@VirginAmerica Congrats on winning the @Travel...,NaN,2015-02-23 13:30:10 -0800,"New York, NY",Pacific Time (US & Canada)


In [15]:
df = df[['airline_sentiment', 'text_x', 'text_y']]
df.drop_duplicates(inplace=True)
len(df)

10259

In [16]:
df['airline_sentiment'].value_counts()

negative    6421
neutral     2126
positive    1712
Name: airline_sentiment, dtype: int64

# Considerar solo algunos tweets


In [17]:
df_1 = df.loc[(df['airline_sentiment'] == 'negative')]
df_2 = df.loc[(df['airline_sentiment'] == 'positive') | (df['airline_sentiment'] == 'neutral')]


In [18]:
print(len(df_1))
print(len(df_2))

6421
3838


In [19]:
del df

**Se usará la columna 'texto completo' para extraer los datos de los tweets**

In [20]:
pd.set_option('display.max_colwidth', -1)
df_1['texto_completo'] =  df_1['text_x'].str.lower()
df_2['texto_completo'] =  df_2['text_x'].str.lower()


<ipython-input-20-2202e51a52d4>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


# remove links

In [21]:
df_1['texto_completo'].replace({"http\S+": '<linkremoved>'}, inplace=True, regex=True)
df_2['texto_completo'].replace({"http\S+": '<linkremoved>'}, inplace=True, regex=True)


In [22]:
df_1['texto_completo'].replace({"<link removed>": '<linkremoved>'}, inplace=True, regex=True)
df_2['texto_completo'].replace({"<link removed>": '<linkremoved>'}, inplace=True, regex=True)



# remove usernames

In [23]:
df_1['texto_completo'].replace({"@[^\s]+": '<usernameremoved>'}, inplace=True, regex=True)
df_2['texto_completo'].replace({"@[^\s]+": '<usernameremoved>'}, inplace=True, regex=True)

# remove duplicates

In [24]:
df_1.drop_duplicates(['texto_completo'],keep='first', inplace=True)
df_2.drop_duplicates(['texto_completo'],keep='first', inplace=True)

In [25]:
print("El numero de tweets no duplicados es", len(df_1)) #numero de tweets no duplicados
print("El numero de tweets no duplicados es", len(df_2)) #numero de tweets no duplicados

El numero de tweets no duplicados es 6417
El numero de tweets no duplicados es 3740


# remove digits, puntuactions, symbols

In [26]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [27]:
punctuation+='¡¿'

In [28]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~¡¿'

In [29]:
#Remove digits and puntuaction
remove_digits = str.maketrans(digits, ' '*len(digits))#remove_digits = str.maketrans('', '', digits)
remove_punctuation = str.maketrans(punctuation, ' '*len(punctuation))#remove_punctuation = str.maketrans('', '', punctuation)
remove_hashtags_caracter = str.maketrans('#', ' '*len('#'))
#las palabras de los hashtag se mantiene, pero no el simbolo. 

# Ojo, en Topic modeling no remuevo tildes. Solo lo hago en word embeddings

In [30]:
import unidecode

data_1 = []
for tweet in df_1['texto_completo']:
    tweet = tweet.translate(remove_digits)
    #tweet = tweet.lower() it wasn't a good idea,, we lost a lot of
    tweet = tweet.translate(remove_punctuation)
    tweet = tweet.translate(remove_hashtags_caracter)
    tweet = tweet.lower()
    #tweet = unidecode.unidecode(tweet)  #esta linea se hacia en word embeddings
    #tweet = tweet.strip().split()
    #filtered_words = [word for word in tweet if word not in stopWords]
    #corpus[id_tweet]= filtered_words
    #id_tweet+=1
    data_1.append(tweet)


In [31]:

data_2 = []
for tweet in df_2['texto_completo']:
    tweet = tweet.translate(remove_digits)
    #tweet = tweet.lower() it wasn't a good idea,, we lost a lot of
    tweet = tweet.translate(remove_punctuation)
    tweet = tweet.translate(remove_hashtags_caracter)
    tweet = tweet.lower()
    #tweet = unidecode.unidecode(tweet)  #esta linea se hacia en word embeddings
    #tweet = tweet.strip().split()
    #filtered_words = [word for word in tweet if word not in stopWords]
    #corpus[id_tweet]= filtered_words
    #id_tweet+=1
    data_2.append(tweet)


In [32]:
print(len(data_1))
print(len(data_2))

6417
3740


#  Twitter tokenizer

In [33]:
from nltk.tokenize import TweetTokenizer

In [34]:
tknzr = TweetTokenizer()
def sent_to_words(sentences):
    for sentence in sentences:
        yield(tknzr.tokenize(sentence))


In [35]:
data_words_1 = list(sent_to_words(data_1))
data_words_2 = list(sent_to_words(data_2))


# Create brigram and trigram models

In [36]:
# Build the bigram and trigram models
#quizas el min count es muy bajo
#documentaicon de bigramas 
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#3importpackages
min_count = 5#int(len(df)*0.03)
bigram_1 = gensim.models.Phrases(data_words_1, min_count=min_count) # higher threshold fewer phrases.
trigram_1 = gensim.models.Phrases(bigram_1[data_words_1], min_count =min_count)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod_1 = gensim.models.phrases.Phraser(bigram_1)
trigram_mod_1 = gensim.models.phrases.Phraser(trigram_1)



In [37]:

min_count = 5#int(len(df)*0.03)
bigram_2 = gensim.models.Phrases(data_words_2, min_count=min_count) # higher threshold fewer phrases.
trigram_2 = gensim.models.Phrases(bigram_2[data_words_2], min_count =min_count)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod_2 = gensim.models.phrases.Phraser(bigram_2)
trigram_mod_2 = gensim.models.phrases.Phraser(trigram_2)



# Remove Stopwords, Make Bigrams and Lemmatize

In [38]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]


In [39]:
def make_bigrams(texts, bigram_mod):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts, bigram_mod,trigram_mod):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [40]:
#%pprint

In [41]:
# Remove Stop Words
data_words_nostops_1 = remove_stopwords(data_words_1)#0:33
# Remove Stop Words
data_words_nostops_2 = remove_stopwords(data_words_2)#0:33


In [42]:
#%pprint


In [43]:
# Form Bigrams
data_words_bigrams_1 = make_bigrams(data_words_nostops_1, bigram_mod_1)
# Form Trigrams

data_words_trigrams_1 = make_trigrams(data_words_bigrams_1, bigram_mod_1, trigram_mod_1)


In [44]:
# Form Bigrams
data_words_bigrams_2 = make_bigrams(data_words_nostops_2, bigram_mod_2)
# Form Trigrams

data_words_trigrams_2 = make_trigrams(data_words_bigrams_2, bigram_mod_2,trigram_mod_2 )


# Inicializar en el idioma correspondiente

In [45]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
if idioma == 'english':
    !python -m spacy download en_core_web_sm
    print("IDIOMA ACTUAL : ENGLISH")
elif idioma == 'spanish':
    !python -m spacy download es
    print("IDIOMA ACTUAL : SPANISH")
else:
    print("Error!!!!!!!")

[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
IDIOMA ACTUAL : ENGLISH


In [46]:
if idioma == 'english':
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    print("IDIOMA ACTUAL : ENGLISH")
elif idioma == 'spanish':
    nlp = spacy.load('es', disable=['parser', 'ner'])
    print("IDIOMA ACTUAL : SPANISH")
else:
    print("ERROR!!")

IDIOMA ACTUAL : ENGLISH


In [47]:
# # Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized_1 = lemmatization(data_words_trigrams_1, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
data_lemmatized_2 = lemmatization(data_words_trigrams_2, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])



# Create dictionary and corpus for topic modeling

In [48]:
# The two main inputs to the LDA topic model are the dictionary(id2word) and the corpus
# Create Dictionary
id2word_1 = corpora.Dictionary(data_lemmatized_1)

# Create Corpus
texts_1 = data_lemmatized_1

# Term Document Frequency
corpus_1 = [id2word_1.doc2bow(text) for text in texts_1]



In [49]:
# Create Dictionary
id2word_2 = corpora.Dictionary(data_lemmatized_2)

# Create Corpus
texts_2 = data_lemmatized_2

# Term Document Frequency
corpus_2 = [id2word_2.doc2bow(text) for text in texts_2]


In [50]:
print(id2word_1.num_docs) #NUMERO DE TWEETS PROCESADOS
print(id2word_2.num_docs)

6417
3740


In [51]:
print("El vocabulary size es: ",str(len(id2word_1)))
print("El vocabulary size es: ",str(len(id2word_2)))


El vocabulary size es:  4480
El vocabulary size es:  3013


# Load and select topic model (ya fueron procesados antes) Seleccionar el mejor modelo de los ya realizados

In [52]:
#coherence_and_model_lists_airlines_data_scenario_1.pkl
import pickle
with open('coherence_and_model_lists_airlines_data_scenario_2_negative_tweets.pkl', 'rb') as handle:
    dict_models_1 = pickle.load(handle)
    
with open('coherence_and_model_lists_airlines_data_scenario_2_positive_neutral_tweets.pkl', 'rb') as handle:
    dict_models_2 = pickle.load(handle)


In [53]:
lda_negative_tweets = dict_models_1['model_list'][5]
lda_positive_tweets = dict_models_2['model_list'][4]

In [54]:
lda_positive_tweets.num_topics

5

In [55]:
lda_model_1 = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(lda_negative_tweets)
lda_model_2 = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(lda_positive_tweets)

### [ Optional ] Create a new topic modeling. It is not necessary anymore

In [56]:
#Instructions about how to install mallet are available here: http://mallet.cs.umass.edu/download.php

'''
Windows installation: After unzipping MALLET, set the environment variable %MALLET_HOME% to point to the MALLET directory.
In all command line examples, substitute bin\mallet for bin/mallet.
'''
import os
from gensim.models.wrappers import LdaMallet


path_to_mallet_binary = "C:\\mallet-2.0.8\\bin\\mallet"
os.environ.update({'MALLET_HOME':r'C:\mallet-2.0.8'}) #OJO!, por alguna razon mallet solo puede estar disponible en esa carpeta


In [57]:
lda_model_2 = LdaMallet(path_to_mallet_binary, corpus=corpus_2, num_topics=5, id2word=id2word_2)

In [58]:
#convert lda mallet model to lda gensim model
lda_model_2 = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(lda_model_2)

# Visualize results using TopicVisExplorer

## Scenario 2

### Get most relevant documents

In [59]:
#with this code we get the full matrix of topic-documents contribution
matrix_documents_topic_contribution_1, _ = lda_model_1.inference(corpus_1)
matrix_documents_topic_contribution_1 /= matrix_documents_topic_contribution_1.sum(axis=1)[:, None]
matrix_documents_topic_contribution_1 = pd.DataFrame(matrix_documents_topic_contribution_1)
matrix_documents_topic_contribution_1.head()

,0,1,2,3,4,5
0,0.201657,0.175404,0.151515,0.151515,0.168394,0.151515
1,0.205509,0.151515,0.168145,0.153067,0.170249,0.151515
2,0.148810,0.184524,0.166667,0.184524,0.166667,0.148810
3,0.148810,0.166667,0.202381,0.184524,0.148810,0.148810
4,0.184619,0.166667,0.148810,0.191800,0.148810,0.159296


In [60]:
contents_1 = pd.Series(df_1['texto_completo']).reset_index(drop=True)

In [61]:
matrix_documents_topic_contribution_1 = pd.concat([matrix_documents_topic_contribution_1, contents_1], axis=1)


In [62]:
matrix_documents_topic_contribution_1.head()

,0,1,2,3,4,5,texto_completo
0,0.201657,0.175404,0.151515,0.151515,0.168394,0.151515,<usernameremoved> i called a 3-4 weeks ago about adding 3 flights from 2014 to my elevate...they still haven't shown up...help!
1,0.205509,0.151515,0.168145,0.153067,0.170249,0.151515,<usernameremoved> is anyone doing anything there today? website is useless and no one is answering the phone.
2,0.148810,0.184524,0.166667,0.184524,0.166667,0.148810,<usernameremoved> amazing to me that we can't get any cold air from the vents. #vx358 #noair #worstflightever #roasted #sfotobos
3,0.148810,0.166667,0.202381,0.184524,0.148810,0.148810,<usernameremoved> it was a disappointing experience which will be shared with every business traveler i meet. #neverflyvirgin
4,0.184619,0.166667,0.148810,0.191800,0.148810,0.159296,<usernameremoved> status match program. i applied and it's been three weeks. called and emailed with no response.


In [63]:
#with this code we get the full matrix of topic-documents contribution
matrix_documents_topic_contribution_2, _ = lda_model_2.inference(corpus_2)
matrix_documents_topic_contribution_2 /= matrix_documents_topic_contribution_2.sum(axis=1)[:, None]

In [64]:
matrix_documents_topic_contribution_2 = pd.DataFrame(matrix_documents_topic_contribution_2)
matrix_documents_topic_contribution_2.head()

,0,1,2,3,4
0,0.240741,0.185185,0.185185,0.185185,0.203704
1,0.200000,0.200000,0.200000,0.200000,0.200000
2,0.203704,0.185185,0.185185,0.203704,0.222222
3,0.207547,0.188679,0.207547,0.207547,0.188679
4,0.178571,0.180713,0.230935,0.178571,0.231209


In [65]:
contents_2 = pd.Series(df_2['texto_completo']).reset_index(drop=True)

In [66]:
matrix_documents_topic_contribution_2 = pd.concat([matrix_documents_topic_contribution_2, contents_2], axis=1)


In [ ]:
matrix_documents_topic_contribution_2.head()

### Topic similarity emtric

In [ ]:
# Choose the # top keywords and # top documents a considerar en la metrica

topn_terms = 20
topk_documents = 20
relevance_lambda = 0.6 

ruta_word_embedding = '../data/embedding_english_europe_northamerica_word2vec_300dimensions_cbow_trim3_epoch50.model'
word_embedding_model = gensim.models.Word2Vec.load(ruta_word_embedding)

In [ ]:
import sys
sys.path.insert(0,'..')


In [ ]:
import topicvisexplorer
import importlib
importlib.reload(topicvisexplorer)
warnings.filterwarnings('ignore')
vis = topicvisexplorer.TopicVisExplorer("borrar_nombre")
topic_similarity_matrix_multicorpora = vis.calculate_topic_similarity_on_multi_corpora(word_embedding_model, lda_model_1,lda_model_2, corpus_1,corpus_2, id2word_1,id2word_2, matrix_documents_topic_contribution_1,matrix_documents_topic_contribution_2, topn_terms, topk_documents, relevance_lambda)

In [ ]:
#import topicvisexplorer
#import importlib
importlib.reload(topicvisexplorer)

vis = topicvisexplorer.TopicVisExplorer("borrar_nombre")
vis.prepare_multi_corpora( lda_model_1,lda_model_2, corpus_1, corpus_2, id2word_1,id2word_2,  matrix_documents_topic_contribution_1, matrix_documents_topic_contribution_2, topic_similarity_matrix_multicorpora)
#save data
vis.save_multi_corpora_data("../models_output/multi_corpora_data_airlines_dataset.pkl")

# Check topic similarity metric baseline

In [67]:
#import gensim_helpers 
#from _prepare import prepare, js_PCoA, PreparedData, _pcoa
topn_terms = 20
relevance_lambda = 0.6 

ruta_word_embedding = '../data/embedding_english_europe_northamerica_word2vec_300dimensions_cbow_trim3_epoch50.model'
word_embedding_model = gensim.models.Word2Vec.load(ruta_word_embedding)

In [68]:
import sys
sys.path.insert(0,'..')


In [69]:
import topicvisexplorer
import importlib
importlib.reload(topicvisexplorer)
warnings.filterwarnings('ignore')
vis = topicvisexplorer.TopicVisExplorer("borrar_nombre")
topic_similarity_matrix_multicorpora_metric_baseline = vis.calculate_topic_similarity_on_multi_corpora_metric_baseline(word_embedding_model,  lda_model_1,lda_model_2, corpus_1, corpus_2, id2word_1,id2word_2, relevance_lambda, topn_terms)



Using Prepared Data script - sort_topics False
Using Prepared Data script - sort_topics False
Not found {'jetblue'}
Not found {'rebooke'}
Not found {'jetblue'}
Not found {'rebooke'}
Not found {'jetblue'}
Not found {'rebooke'}
Not found {'jetblue'}
Not found {'rebooke'}
Not found {'rebooke', 'jetblue'}
Not found {'rebooke'}
Not found {'rebooke'}
Not found {'rebooke'}
Not found {'rebooke'}
Not found {'jetblue'}
Not found {'rebooke'}


In [70]:
vis.prepare_multi_corpora( lda_model_1,lda_model_2, corpus_1, corpus_2, id2word_1,id2word_2,  matrix_documents_topic_contribution_1, matrix_documents_topic_contribution_2, topic_similarity_matrix_multicorpora_metric_baseline)


Using Prepared Data script - sort_topics False
Using Prepared Data script - sort_topics False


In [71]:
vis.save_multi_corpora_data("../models_output/multi_corpora_data_airlines_dataset_baseline_metric.pkl")

Multi corpora data saved sucessfully
